In [1]:
###Testing infersent to get principal text entailment sentences


In [1]:
import nltk
nltk.download('punkt')
import time
import tqdm
import math
import numpy as np

from collections import Counter
from matplotlib import pyplot as plt 
from operator import itemgetter 
from nltk.tokenize import word_tokenize
from torch.autograd import Variable


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zaiem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import torch
# if you are on GPU (encoding ~1000 sentences/s, default)
#infersent = torch.load('./encoder/infersent.allnli.pickle')
# if you are on CPU (~40 sentences/s)
infersent = torch.load('./encoder/infersent.allnli.pickle', map_location=lambda storage, loc: storage)


C:\Anaconda3\lib\site-packages\torch\serialization.py:425: SourceChangeWarning: source code of class 'models.BLSTMEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Anaconda3\lib\site-packages\torch\serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [4]:
glove_path= "./dataset/GloVe/glove.840B.300d.txt"
infersent.set_glove_path(glove_path)


In [12]:
from models import InferSent
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))


ImportError: cannot import name 'InferSent'

In [ ]:
#You'll have to load the sentences from whose you want to extract keywords

In [5]:
%%time
infersent.build_vocab(sentences, tokenize=True)


NameError: name 'sentences' is not defined

In [34]:
len(sentences)

730658

In [33]:
%%time
embeddings = infersent.encode(sentences[0:10000], tokenize=True)


CPU times: user 6min 45s, sys: 7.58 s, total: 6min 52s
Wall time: 1min 41s


In [1]:
#Main keyword extraction function, it encodes the sentence with Infersent than takes the most important words
# according to the maxpooling layer
def new_visualize(sent, infersent) :
    sent = word_tokenize(sent)
    if (len(sent)<7) : 
        number = 3 
    elif (len(sent)<10) : 
        number = 4 
    elif (len(sent)<13) :
        number = 5 
    else :
        number = 6 
    sent = [['<s>'] + [word for word in sent if word in infersent.word_vec] +
            ['</s>']]

    if ' '.join(sent[0]) == '<s> </s>':
        import warnings
        warnings.warn('No words in "{0}" have glove vectors. Replacing \
                       by "<s> </s>"..'.format(sent))
    batch = Variable(infersent.get_batch(sent), volatile=True)
    
    output = infersent.enc_lstm(batch)[0]
    output, idxs = torch.max(output, 0)
    # output, idxs = output.squeeze(), idxs.squeeze()
    idxs = idxs.data.cpu().numpy()
    argmaxs = [np.sum((idxs == k)) for k in range(len(sent[0]))]
    important_words_idx= (-np.array(argmaxs)).argsort()[:number].tolist()
    important_words= [sent[0][i] for i in important_words_idx]
    return important_words

In [2]:
def get_words_from_function(arr, number, sentence) : 
    sentence = word_tokenize(sentence )
    idxs= (-np.array(arr)).argsort()[:number].tolist()
    print(idxs)
    return sentence 
    for i in idxs : 
        print (sentence[i-1])


In [173]:
# Test on the first ten sentences 
counters = []
for i in tqdm.tqdm(range(len(sentences[0:10]))) : 
    print(sentences[i])
    counters.append(new_visualize(sentences[i], infersent))
    


  0%|          | 0/10 [00:00<?, ?it/s]

What are your favorite and least favorite alcoholic beverages?



 20%|██        | 2/10 [00:00<00:01,  5.12it/s]

['alcoholic', 'beverages', 'your', 'favorite', '?']
Somewhere backstage, an argument was raging.
['raging', 'backstage', 'argument', 'Somewhere']
The Irish Free State became reality in 1922.



 30%|███       | 3/10 [00:00<00:01,  5.11it/s]

...

['french', 'plate', 'fries', 'fish', 'fried', 'bed']
I've written out the mathematical proposition in English that's easier to understand.



 90%|█████████ | 9/10 [00:02<00:00,  4.22it/s]

['mathematical', 'English', 'easier', "'ve", 'proposition', 'written']
The alcohol issues involved dangerous drinking.
['drinking', 'dangerous', 'issues', 'alcohol']
A plate of food with vegetables and meat.



100%|██████████| 10/10 [00:02<00:00,  4.31it/s]


['plate', 'meat', 'vegetables', 'food']


In [180]:
expanders = []
for i in tqdm.tqdm(range(len(sentences))) : 
    expanders.append(new_visualize(sentences[i], infersent))
    


100%|██████████| 730658/730658 [54:12:58<00:00,  3.74it/s]   


In [181]:
# Print random expansions  
expanders[10000:10010]


[['television', 'big', 'screen', 'room', 'sitting', 'stand'],
 ['Acid', 'Rain', 'state-of-the-art', 'data', 'cap', 'administered'],
 ['<s>', 'i', 'used', 'before', '</s>'],
 ['ban', 'products', 'China', 'India', 'Should'],
 ['avoid', 'thinking', 'lots'],
 ['suitcase', 'bed', 'sitting', 'woman', 'on'],
 ['restaurant', 'street', 'standing', 'sign', 'on', 'A'],
 ['depot', 'clock', 'blue', 'travels', 'atop', 'sits'],
 ['Church', 'spire', 'capped', 'mountains', 'tops'],
 ['War', 'III', 'break', 'World']]